# 1. Importing Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# 2. Reading Data into Dataframe

Since the dataset is not a CSV file, but rather an XLSX file, the `openpyxl` library is required to convert the dataset into Pandas Dataframe

In [ ]:
!pip install openpyxl

In [ ]:
df = pd.read_excel('1730285881-Airbnb_Open_Data.xlsx')

# 3. Data Preprocessing

In [ ]:
df.head()

In [ ]:
df.sample(5)

In [ ]:
df.info()

## Looking for duplicates

In [ ]:
df.duplicated().value_counts()

In [ ]:
df.duplicated().sum()

**541 duplicate** entries were discovered in the given dataset. Therefore, all duplicates have been dropped.

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.duplicated().sum()

## Checking null values

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace = True)

## Inspecting Important Columns

In [ ]:
df['neighbourhood group'].value_counts()

In the column *neighbourhood group*, there are **two errors** relating to Manhattan and Brooklyn. Thus, I'll replace them with the correct value.

In [ ]:
df['neighbourhood group'] = df['neighbourhood group'].replace({'brookln':'Brooklyn', 'manhatan':'Manhattan'})

In [ ]:
df['country'].value_counts()

In [ ]:
df['room type'].value_counts()

In [ ]:
df['price'].sample(20)

In [ ]:
df['service fee'].sample(20)

The values in price and service fee, have dtype **float64** and are purely numerical thus no formatting is required. However their names must contain the currency.

In [ ]:
df.rename(columns = {'price':'price_$', 'service fee':'service_fee_$'}, inplace = True)

In [ ]:
df['house_rules'].head()

The column *house_rules* is irrelevant to our analysis and *license* contains only 2 entries thus they are dropped.

In [ ]:
df.drop(['house_rules', 'license'], axis=1, inplace=True

In [ ]:
df['availability 365'].sample(20)

In [ ]:
plt.boxplot(df['availability 365'])

In [ ]:
#removing the outliers i.e values greater than 500 as seen in the plot
df = df.drop(df[df['availability 365'] > 500].index)

## Converting features into appropriate dtypes

In [ ]:
df.info()

In [ ]:
df['last review'].sample(5)

In [ ]:
df['id'] = df['id'].astype(str)
df['host id'] = df['host id'].astype(str)
df['Construction year'] = df['Construction year'].astype(int)

# 4. Exploratory Data Analysis

In [ ]:
df.describe()

## Types of Rooms

> What are the different propery types in the AirBnB open Dataset?

In [ ]:
property_types = df['room type'].value_counts().to_frame()
property_types

In [ ]:
room_type_bar = plt.bar(property_types.index, property_types.loc[:, 'count'])
plt.bar_label(room_type_bar, labels = property_types.loc[:, 'count'], padding = 4)
plt.ylim([0, 50000])
plt.xlabel('Room Type')
plt.ylabel('Room Type count')
plt.title('Property Types and their Count')

There are four distinct room or property types as per the given AirBnB Open Dataset which are

- Entire home/apt
- Private Room
- Shared Room
- Hotel Room

## Neighbourhood Group with Highest Listing

> Which Neighbourhood Group has the Highest Listing?

In [ ]:
hood_group = df['neighbourhood group'].value_counts().to_frame()
hood_group

In [ ]:
hood_group_bar = plt.bar(hood_group.index, hood_group.loc[:, 'count'])
plt.bar_label(hood_group_bar, labels = hood_group.loc[:, 'count'], padding = 4)
plt.ylim([0, 40000])
plt.xlabel('Neighbourhood Groups')
plt.ylabel('Number of Listings')
plt.xticks(rotation = 45)
plt.title('Neighbourhood Group with the Highest Listing')

Brooklyn has the Highest Number of Listings, with Manhattan being a close second. Following next are Queens, Bronx, and Staten Island.

## Neighbourhood Group with highest Average Price

> Which Neighbourhood Group has highest Average Price?

In [ ]:
avg_price = df.groupby('neighbourhood group')['price_$'].mean().sort_values(ascending = False).to_frame()
avg_price

In [ ]:
avg_price_bar = plt.bar(avg_price.index, avg_price.loc[:, 'price_$'])
plt.bar_label(avg_price_bar, labels = round(avg_price.loc[:, 'price_$'], 2), label_type = 'edge', padding = 4)
plt.ylim([0, 700])
plt.xlabel('Neighbourhood Group')
plt.ylabel('Average Price per Listing ($)')
plt.xticks(rotation = 45)
plt.title('Average Price per Listing ($) in each Neighbourhood Group')

There is no major difference among the Average Price per Listing ($) of each Neighbourhood group. Regardless, Queens has the  highest average price per listing, followed by Bronx, Brooklyn, Manhattan, and Staten Island.

## Relation between Construction Year and Price of Property

> Is there a Relation between Construction Year and Price of Property?

In [ ]:
df.groupby(df['Construction year'])['price_$'].mean().to_frame().plot()
plt.xlabel('Construction year')
plt.ylabel('Average Price ($)')
plt.title('Average Price ($) for Properties in each Construction Year')

The line plot shows a noticeable downward trend as the Construction Year Progresses, with periodic lows and highs.

## Top 10 Hosts

> Which are the Top 10 Hosts by Calculated Host Listings Counts?

In [ ]:
hosts = df.groupby('host name')['calculated host listings count'].sum().sort_values(ascending = False).nlargest(10).to_frame()
hosts

In [ ]:
hosts_bar = plt.bar(hosts.index, hosts.loc[:, 'calculated host listings count'])
plt.bar_label(hosts_bar, label = hosts.loc[:, 'calculated host listings count'], label_type = 'edge', padding = 3)
plt.xlabel('Hosts Name')
plt.ylabel('Caculated Host Listings Count')
plt.xticks(rotation = 80)
plt.ylim([0, 120000])
plt.title('Top 10 Hosts by Calculated Host Listings Counts')

The Highest Calculated Host Listing belonged to Sonder (NYC) with 111921 Listings with the second highest, Karen trailing far behind at 42410 listings. The remaining contenders follow each other closely.

## Relation between Verified Identities and Positive Reviews

> Are hosts with verified identities more likely to receive positive reviews?

In [ ]:
review = df.groupby('host_identity_verified')['review rate number'].mean().sort_values(ascending = False).to_frame()
review

In [ ]:
review_bar = plt.bar(review.index, review.loc[:, 'review rate number'])
plt.bar_label(review_bar, labels = round(review.loc[:, 'review rate number'], 2), padding = 4)
plt.ylim([0, 4])
plt.xlabel('Host Verification Status')
plt.ylabel('Average Review Rate Number')
plt.title('Average Review Rate for each verification statistics')

In [ ]:
base_color = sns.color_palette()[0]
sns.boxplot(data = df, x = 'host_identity_verified', y = 'review rate number', color = base_color)
plt.xlabel('Host Verification Status')
plt.ylabel('Review Rate Number')
plt.title('Average Review Rate for each verification status')

There is no visible relation between Average Review Rate and the Verification Status. Hosts have similar Average Review Rates irrespective of thier verification status. Thus, Verification Status does not influence the Review Rate. Precisely speaking, Verified Hosts have a marginally higher Average Review Rate of 3.28 as compared to 3.27 of Unverified Hosts, but it is too small to represent a relation.

## Correlation between Price and Service Fee of Listing

> Is there a Correlation between Price and Service Fee of Listing?

In [ ]:
df['price_$'].corr(df['service_fee_$'])

In [ ]:
sns.regplot(df, x = 'price_$', y = 'service_fee_$')
plt.xlabel('Price ($)')
plt.ylabel('Service Fee ($)')
plt.title('A Regression Plot showing the correlation between Price and Service Fee of a Listing')

A Pearson Correlation Coefficent of 0.99~ between the Price and Service Fee of Listings indicates a h  igh ly robust postive correlation between these two variables. The Regression Line Plot shows the increase of Service Fee with increase of Listing Price, represented in the form of a straight diagonal rising towards the right.

## Average Review Rate Number

> What is the average review rate number for listings, and does it vary based on the neighbourhood group and room type?

In [ ]:
ARRN = df.groupby(['neighbourhood group', 'room type'])['review rate number'].mean().to_frame()
ARRN

In [ ]:
plt.figure(figsize = [12, 10])
sns.barplot(data = df, x = 'neighbourhood group', y = 'review rate number', hue = 'room type')
plt.xlabel('Neighbourhood Group')
plt.ylabel('Average Review Rate')
plt.title('Average Review Rate for each Room or Property Type in each Neighbourhood Group')

In each of the three Neighbourhood Groups containing Hotel Groups i.e. Brooklyn, Manhattan, and Queens, Hotel Rooms have a Higher Review Rate as compared to the other Property Types.

## Hosts with Higher Calculated Listings Counts and likelihood of Higher Availability throughout the year

> Are hosts with a higher calculated host listings count more likely to maintain a higher availability throughout the year?

In [ ]:
sns.regplot(df, x = 'calculated host listings count', y = 'availability 365')
plt.xlabel('Calculated Host Listings')
plt.ylabel('Availability 365')
plt.title('A Regression Plot of the Relationship between Calculated Host Listings Count and Availability 365')

In [ ]:
df['calculated host listings count'].corr(df['availability 365'])

With a Pearson Correlation Coefficient value of 0.136~ between Calculated Host Listings and Availbility 365, it is indicated that there is a weak positive correlation between the two variables.

# Conclusion

In conclusion, this project has provided a **comprehensive analysis of the New York City AirBnB Dataset**, shedding light on the various aspects of the short-term lodging market. Through *Data Wrangling, Exploratory Data Analysis (EDA), and Interpretation of Summary Statistics*, we've uncovered valuable insights into **listing distribution, pricing dynamics, host, and review analysis**.

Key findings include the dominance in counts of `Entire home/apt` listings, the variability in `Listing Counts` across neighbourhood groups, and the downward trend between `Property Construction Year` and `Price`. Additionally, the analysis highlighted the significance of `Verified Host status` on `Review Rates`, as well as the strong correlation between `Listing Price` and `Service Fee`.

Furthermore, conducting sentiment analysis on guest reviews to understand factors driving customer satisfaction and preferences could inform targeted marketing strategies and product improvements for AirBnB. Lastly, expanding the analytics to include predictive modelling techniques, such as regression or machine learning algorithms, could enable forecasting of listing demand, pricing trends, and customer behaviour, facilitating strategic decision-making for stakeholders in the short-term lodging industry.